In [ ]:
import serial
from tkinter import *
import threading
from datetime import datetime
import math

print('Connecting...')
ser = serial.Serial('COM5', 250000, timeout=1000)
if ser.isOpen():
    print('Connected!')

on = True
loop_active = True
data_receive = True

data_save = None
save_flag = -1
test_trial = 1

low_degree = 0
high_degree = 0

prev_touch = 0
touch_mode = 0

swing_stat = 0
swing_cnt = 0


class Upd(threading.Thread):

    def __init__(self, tk_root):
        self.root = tk_root
        threading.Thread.__init__(self)
        self.start()

    def run(self):
        global loop_active, save_flag, data_save, prev_touch, touch_mode, swing_stat, swing_cnt, low_degree, high_degree, angle_1
        temp_cnt = 0
        time_prev = 0
        while loop_active:
            if data_receive:
                x = ser.readline()
                x = x.split()

                if len(x) > 7:
                    if save_flag == 1:  # 녹화 중일 때
                        XpYpZ = (float(x[5]) ** 2 + float(x[6]) ** 2 + float(x[7]) ** 2) ** 0.5
                        try:
                            angle_1 = math.atan(float(x[2]) / float(x[4])) * 180 / math.pi
                            # 자이로 계산 값 : (역탄젠트(가속도 x/가속도 z)) *180 / PI을 angle_1에 넣는다.
                            angle_1 = abs(angle_1)  # 무조건 양수로 받음 (스윙과 백스윙 동시판단)

                        except ZeroDivisionError:
                            print("90도를 벗어났습니다.")
                        # try: except 문으로 예외처리를 하여 데이터 수집 시 범위가 벗어나는 오류가 나도 멈추지 않게 작성
                        lb_xyz.configure(text="%6.2f" % angle_1)  # x축의 회전각도(roll)을 레이블을 통해 표시
                        lb_gt.configure(text="%6.2f" % XpYpZ)  # 자이로 값을 레이블을 통해 표시
                        if swing_stat == 0:  # 스윙 상태가 아닐 때
                            if XpYpZ > 80:  # 가속도 계산 값이 50 이상일 때
                                swing_stat = 1  # 스윙이라 판단
                                time_prev = datetime.now()  # 현재시간을 저장
                        elif swing_stat == 1:  # 스윙 중 일때
                            if XpYpZ < 30:  # 가속도 값이 다시 내려오면
                                time_now = datetime.now()  # 현재시간을 저장
                                elapsed2 = (time_now - time_prev).total_seconds()  # 시간 차이 계산
                                if elapsed2 > 1:  # 시간의 차이가 1초가 이상나면 (진동, 미세한 스윙 등 제거)
                                    temp_cnt = swing_cnt  # 이전 스윙 횟수를 저장한다.
                                    swing_cnt += 1  # 스윙 횟수를 1 늘린다.
                                    lb_sw.configure(text="%d번 스윙모션을 하셨습니다" % swing_cnt)
                                    swing_stat = 0  # 스윙 상태를 0으로 변경
                                    if swing_cnt % 2 == 1:  # 홀수일경우 백스윙으로 판단
                                        if high_degree > float(angle_1) > low_degree:
                                            # 입력받은 High 값과 Low 값 사이에 roll 값이 있으면
                                            answer = "좋은 백스윙입니다."
                                            ser.write(b'8')  # 부저 울리기
                                            lb_answer.configure(text=answer)
                                        else:
                                            answer = "좋지 않은 백스윙입니다."
                                            ser.write(b'9')
                                            lb_answer.configure(text=answer)

                                    elif swing_cnt % 2 == 0:   # 짝수일 경우 스윙으로 판단
                                        if high_degree > float(angle_1) > low_degree:
                                            # 입력받은 High 값과 Low 값 사이에 roll 값이 있으면
                                            answer = "좋은 스윙입니다."
                                            ser.write(b'8')  # 부저 울리기
                                            lb_answer.configure(text=answer)
                                        else:
                                            answer = "좋지 않은 스윙입니다."
                                            ser.write(b'9')
                                            lb_answer.configure(text=answer)

                    if int(x[1]) == 0:  # 터치센서가 클릭되지 않았을 때
                        if prev_touch == 1:  # 이전에 터치센서를 누른적이 있으면 (눌렀다 땠을 때)
                            touch_mode += 1  # 터치모드를 1증가
                    if touch_mode % 2 == 1:  # 터치모드가 홀수일 때 (눌렀다 때면 1, 다시 눌렀다 때면 2(정지))
                        rec_start()  # 녹화 시작
                    elif touch_mode % 2 == 0:  # 터치모드가 짝수일 때
                        if touch_mode != 0:  # 최초 초기값이 아닐 때
                            rec_stop()  # 녹화 중지

                    prev_touch = int(x[1])  # 터치 코드 실행 후 이전 터치값 = 터치센서

                    if swing_cnt != temp_cnt:  # 스윙값이 변할 때
                        time_out_now = datetime.now()
                        data_save.writelines(str(time_out_now) + "\t%d" % math.ceil(swing_cnt/2) + "번째 ")  # ceil 올림함수
                        if swing_cnt % 2 == 1:          # swing_cnt 는 모든 스윙마다 (백스윙, 스윙) 증가되기에 2로 나눈다.
                            data_save.writelines("백스윙\n")
                        elif swing_cnt % 2 == 0:
                            data_save.writelines("스윙\n")
                        data_save.writelines("최대 각 : " + str(high_degree) + "\t")
                        data_save.writelines("최소 각 : " + str(low_degree) + "\n")
                        data_save.writelines("사용자의 각도 : %6.2f" % angle_1 + "\n\n")

                        temp_cnt = swing_cnt
                        # 임시 카운트 값을 다시 올려줌

        ser.close()
        self.root.quit()
        self.root.update()


def loop_trigger():
    global data_receive
    if data_receive:
        data_receive = False
        ser.write(b's')
    elif not data_receive:
        data_receive = True
        ser.write(b'r')


def rec_start():
    global data_save, save_flag, test_trial
    data_save = open("test" + str(test_trial) + ".txt", "a")
    save_flag = 1
    lb.configure(text='recording...')


def rec_stop():
    global data_save, save_flag, test_trial
    data_save.close()
    test_trial += 1
    save_flag = 0
    lb.configure(text='completed')


def exitProgram():
    global loop_active
    loop_active = False


win = Tk()


def Cal():
    global low_degree, high_degree
    low_degree = float(eval(entryMin.get()))   # input 함수는 문자열로만 받지만
    high_degree = float(eval(entryMax.get()))  # eval 함수는 숫자를 받을 수 있다.
    lb_low.configure(text="%6.2f" % low_degree)   # 첫번째 값을 low_degree 인수에 저장
    lb_high.configure(text="%6.2f" % high_degree) # 두번째 값을 high_degree 인수에 저장


entryMin = Entry(win, width=10)
entryMin.bind("<Return>")
entryMin.pack()

entryMax = Entry(win, width=10)
entryMax.bind("<Return>")
entryMax.pack()

lb_low = Label(win, text="최소 각도")
lb_low.pack()
lb_high = Label(win, text="최대 각도")
lb_high.pack()

click = Button(win, text="적용하기", command=Cal)
click.pack()

win.title("serial_test")
win.geometry("300x400")
upd = Upd(win)

lb = Label(win, text="waiting")
lb.pack()

bt5 = Button(win, text='exit', command=exitProgram)
bt5.pack()

lb_xyz = Label(win, text="Roll 회전")
lb_xyz.pack()
lb_gt = Label(win, text="Gyro total")
lb_gt.pack()
lb_answer = Label(win, text="자세 결과 값")
lb_answer.pack()
lb_sw = Label(win, text="스윙되지 않았습니다.")
lb_sw.pack()

win.mainloop()
